In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
import pickle
from glob import glob
import os
import yaml
from easydict import EasyDict as edict

In [14]:
import sys

sys.path.append('../')

In [15]:
from model.controlmodel import *
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, ConcatDataset

In [16]:
import pytorch_lightning as pl
from torchmetrics import Accuracy

In [18]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint

In [27]:
from runner import *
from dataset import *

In [30]:
config_file = glob('../config/control_model/control_mlp_mnist.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

train_dataset = datasets.CIFAR10(root='../xor_neuron_data/data',
                                train=True,
                                transform=transform,
                                download=False)

validation_dataset = datasets.CIFAR10(root='../xor_neuron_data/data',
                                train=False,
                                transform=transform,
                                download=False)

In [10]:
train_loader = DataLoader(dataset=train_dataset,
                                 batch_size=16,
                                 shuffle=False)

validation_loader = DataLoader(dataset=validation_dataset,
                                 batch_size=16,
                                 shuffle=False)


In [11]:
class Classifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.model = Control_MLP(config)
        self.acc_fnc = Accuracy()
        
    def forward(self, x, y):
        return self.model(x, y)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits, loss = self(x, y)
        metrics = {'train_loss': loss}
        self.log_dict(metrics)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        logits, loss = self(x, y)
        
        acc = self.acc_fnc(logits, y)
        
        metrics = {'val_acc': acc, 'val_loss': loss}
        self.log_dict(metrics)

    def test_step(self, batch, batch_idx):
        x, y = batch

        logits, loss = self(x)
        acc = self.acc_fnc(logits, y)
        
        metrics = {'test_acc': acc, 'test_loss': loss}
        self.log_dict(metrics)
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.model.parameters(),
            lr=0.001,
            weight_decay=0.00001,
            momentum=0.9,
            nesterov=True,
        )
        
        
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer,
            milestones=[1000],
            gamma=0.001)
        
        return [optimizer], [scheduler]

In [ ]:
control_model = Classifier(config)

In [ ]:
control_model

In [ ]:
trainer = pl.Trainer(max_epochs=3, gpus=0)
trainer.fit(control_model, train_loader, validation_loader)

In [41]:
config.save_dir = './test'
config.exp_dir = './test'
config.dataset.data_path = '../xor_neuron_data/data'

In [42]:
checkpoint = ModelCheckpoint(monitor="val_loss", mode="min", save_last=False, dirpath=config.save_dir,
                                 filename='model_snapshot_best_{epoch:d}')

In [43]:
trainer = Trainer(default_root_dir=config.save_dir,
                          max_epochs=config.train.max_epoch,
                          gpus=0,
                          auto_lr_find=True,
                          # logger=logger,
                          deterministic=True,
                          log_every_n_steps=1,
                          callbacks=[checkpoint])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [44]:
runner = PL_Runner(config)
data = PL_DataModule(config)

In [45]:
trainer.fit(runner, data)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../xor_neuron_data/data/MNIST/raw/train-images-idx3-ubyte.gz to ../xor_neuron_data/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../xor_neuron_data/data/MNIST/raw/train-labels-idx1-ubyte.gz to ../xor_neuron_data/data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../xor_neuron_data/data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../xor_neuron_data/data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../xor_neuron_data/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../xor_neuron_data/data/MNIST/raw



/opt/anaconda3/envs/xor_neuron/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459044803/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


AttributeError: 'PL_Runner' object has no attribute 'lr'